# Setup

## Installing and Importing Libraries

In [ ]:
!apt update
!apt install -yq chromium-chromedriver
!pip install selenium

In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

## Connecting to driver and loading website

In [44]:
# Set up options for Chrome in headless mode
options = webdriver.ChromeOptions()
options.add_argument('--headless') # allows Chrome to run without a graphical user interface
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Start the WebDriver
driver = webdriver.Chrome(options=options)

In [38]:
search_term = 'Rafael Nadal'

try:
    # Step 1: Navigate to the URL
    driver.get('https://www.atptour.com/en/players')

    # Step 2: Find the search bar by ID and input the player's name
    driver.implicitly_wait(5)

    search_bar = driver.find_element(By.ID, 'playerInput')
    search_bar.send_keys(search_term)

    # Step 3: Wait for the dropdown to appear and find the first 'no-padding' class
    driver.implicitly_wait(5)

    dropdown = driver.find_element(By.ID, 'playerDropdown')
    first_option = dropdown.find_element(By.CLASS_NAME,"no-padding")

    # Step 4: Retrieve the link inside the first 'no-padding' class
    link_class = first_option.find_element(By.TAG_NAME,'a')
    link = str(link_class.get_attribute("href"))
    print(link)

finally:
    # Optional: Close the browser window when you are done
    driver.quit()

https://www.atptour.com/en/players/rafael-nadal/n409/overview


In [45]:
try:
    # Step 1: Navigate to the player's URL
    driver.get(link)

    # Step 2: Get birthdate
    driver.implicitly_wait(5)

    birthdate = driver.find_element(By.CLASS_NAME, "table-birthday").text
    print(birthdate)

finally:
    # Optional: Close the browser window when you are done
    driver.quit()

(1986.06.03)
